In [18]:
#Backtracking and Forward Tracking Solver#

In [27]:
import time
import copy
import sys
sys.path.append(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence')
from sudoku_parser import SudokuPuzzle

#load in Sudoku Puzzle#

puzzle = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')
puzzle.set_domains()
puzzle.display()

? 7 9 4 3 ? 1 ? ?
3 ? ? ? ? ? 6 ? ?
8 5 ? ? 6 7 ? 3 ?
? ? ? ? ? 6 ? 8 ?
? ? 5 ? 4 ? 7 ? ?
? 8 ? 5 ? ? ? ? ?
? 9 ? 1 7 ? ? 6 3
? ? 8 ? ? ? ? ? 1
? ? 6 ? 9 4 2 7 ?


In [28]:
#Use Domains for Backtracking and Forward Tracking#

In [29]:
def get_neighbors(row, col):
    neighbors = set()

    #Row and Column Neighbors#
    for i in range(9):
        neighbors.add((row, i))
        neighbors.add((i, col))

    #Box Neighbors#
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            neighbors.add((i, j))

    neighbors.discard((row, col))
    return neighbors


In [30]:
#Heuristic: Get Variables Using Degree#

In [31]:
def select_variable(puzzle: SudokuPuzzle):
   #This is Used To Select the Varaible with the Most Constraints#
    unassigned = puzzle.available_indexes()
    max_degree = -1
    best_cell = None

    for row, col in unassigned:
        degree = 0
        for neighbors_row, neighbors_col in get_neighbors(row, col):
            if puzzle.grid[neighbors_row][neighbors_col] == 0:
                degree += 1
        if degree > max_degree:
            max_degree = degree
            best_cell = (row, col)

    return best_cell


In [32]:
#Backtracking and Forward Checking#

In [33]:
def forward_checker(puzzle: SudokuPuzzle, row, col, value):
    for r, c in get_neighbors(row, col):
        if puzzle.grid[r][c] == 0 and puzzle.domains[r][c]:
            if value in puzzle.domains[r][c]:
                puzzle.domains[r][c].remove(value)
                if not puzzle.domains[r][c]:
                    return False
    return True


In [34]:
def forward_solver(puzzle: SudokuPuzzle) -> bool:
    if not puzzle.available_indexes():
        return True  #Puzzle Solved

    #Select variable using Heuristic (Degree)
    row, col = select_variable(puzzle)
    domain = puzzle.domains[row][col] or list(range(1, 10))

    for value in domain:
        puzzle.set_digit(row, col, value)

        if puzzle.is_valid():
            #Backup of Grid and Domains
            backup_grid = copy.deepcopy(puzzle.grid)
            backup_domains = copy.deepcopy(puzzle.domains)

            #Forward Checking
            if not forward_checker(puzzle, row, col, value):
                # Restore state and continue
                puzzle.grid = backup_grid
                puzzle.domains = backup_domains
                puzzle.set_digit(row, col, 0)
                continue

            if forward_solver(puzzle):
                return True

            #Backtrack
            puzzle.grid = backup_grid
            puzzle.domains = backup_domains
            puzzle.set_digit(row, col, 0)
        else:
            puzzle.set_digit(row, col, 0)

    return False  #No Solution

#Test Print
puzzle = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')
puzzle.set_domains()

if forward_solver(puzzle):
    print("\n✅ Puzzle Solved:")
    puzzle.display()
else:
    print("\n❌ No solution found.")




✅ Puzzle Solved:
6 7 9 4 3 5 1 2 8
3 4 2 9 8 1 6 5 7
8 5 1 2 6 7 9 3 4
4 1 3 7 2 6 5 8 9
9 6 5 3 4 8 7 1 2
2 8 7 5 1 9 3 4 6
5 9 4 1 7 2 8 6 3
7 2 8 6 5 3 4 9 1
1 3 6 8 9 4 2 7 5


In [ ]:
#Detects Early Failures, and Preforms Forward Checking Steps